We need to install (Hugging Face) 🤗 Transformers and 🤗 Datasets.

In [455]:
%%capture
! pip install "datasets" "pytorch-lightning" "wandb" "torcheval" "torchmetrics"

In [456]:
import datasets
import torch
import wandb
import numpy as np

import os
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar

#from torchmetrics import Precision, Recall
#from torchmetrics import F1Score  #, BinaryF1Score
from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryF1Score
#from torchmetrics.classification import F1
from sklearn.metrics import f1_score


from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything

from pytorch_lightning.loggers import WandbLogger
from datasets import load_dataset, load_metric


In [457]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")
print(f"Device: {device}")

Device: cpu


# Loading the Dataset

In [458]:
PATH_DATASETS = "."
BATCH_SIZE = 5

In [459]:
%%capture
train_ds = FashionMNIST(PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor())
#train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

#eval_ds = FashionMNIST(PATH_DATASETS, train=False, download=True, transform=transforms.ToTensor())
#eval_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE)


In [460]:
# taking first 100 sample for speed up computation

# making the dataset binary
for idx in range(len(train_ds)):
    if train_ds.targets[idx] > 5:
        train_ds.targets[idx] = 1
    else:
        train_ds.targets[idx] = 0


In [461]:
class_0_samples_train = sum(1 for label in train_ds.targets if label == 0)
class_1_samples_train = sum(1 for label in train_ds.targets if label == 1)

print("\nNumber of Samples", len(train_ds))
print("Number of Samples for Class 1 in Training Set:", class_1_samples_train)
print("\nNumber of Samples for Class 0 in Training Set:", class_0_samples_train)


Number of Samples 60000
Number of Samples for Class 1 in Training Set: 24000

Number of Samples for Class 0 in Training Set: 36000


In [462]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

In [463]:
print(train_ds.data[0].shape)

torch.Size([28, 28])


# Lightning Model

In [464]:

class Net(pl.LightningModule):
    def __init__(self, num_classes=1):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, num_classes)
        self.num_classes = num_classes

        self.accuracy_metric  = BinaryAccuracy() #(task='multiclass', num_classes=self.num_classes)        #self.f1_metric        = MulticlassF1Score(num_classes=num_classes, average=None)
        self.precision_metric = BinaryPrecision() #(task='multiclass',num_classes=num_classes, average='weighted')
        self.recall_metric    = BinaryRecall() #(task='multiclass',num_classes=num_classes, average='weighted')
        self.f1score_metric   = BinaryF1Score()

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch):
        x, y = batch
        x.clamp_(0, 1)
        y.clamp_(0,1)
        
        loss = F.binary_cross_entropy(self(x).view(-1), y.float())

        predictions = self.forward(x).long().squeeze()
        y = torch.tensor(y, dtype=torch.long)

        predictions.clamp_(0,1)
        

        # Move the accuracy metric to the same device as the input data
        accuracy = self.accuracy_metric.to(x.view(-1).device)
       
        acc       = accuracy(predictions, y)
        precision = self.precision_metric(predictions, y)
        recall    = self.recall_metric(predictions, y)
        #f1_score  = self.f1score_metric(predictions, y)

        
        f1 = f1_score(predictions, y, average='weighted')

        wandb.log({"acc": acc,
                   "loss": loss,
                   "precision": precision,
                   "recall": recall,
                   "f1-score:":f1
                   })
        
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)
    

model = Net()

for name, param in model.named_parameters():
    print(name, param.requires_grad, )

l1.weight True
l1.bias True


# Training With and without WANDB

Trying with and without WANDB

In [465]:
%%capture
personal_key = '8cc162e60d761ed72f9c2298a0ab2b17f40d0f13'
wandb.login(key = personal_key)

#wandb.login()

In [466]:
wandb.init(project="Geometric_Algebra_Transformer",
           name="GATr - wandb template",
           config={
               "learning_rate": 0.001,
               "dataset": "MNIST",
               "epochs": 2,
               "train_batch_size": 256,
               "eval_batch_size": 256
           })


acc,▁
f1-score:,▁
loss,▁
precision,▁
recall,▁
acc,0.8
f1-score:,0.88889
loss,0.61666
precision,0.0
recall,0.0


In [468]:
wandb_logger = WandbLogger(log_model="all")
trainer = Trainer(max_epochs=1, accelerator="auto", logger=wandb_logger)
trainer.fit(model, train_loader)

# mark the run as finished
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name             | Type            | Params
-----------------------------------------------------
0 | l1               | Linear          | 785   
1 | accuracy_metric  | BinaryAccuracy  | 0     
2 | precision_metric | BinaryPrecision | 0     
3 | recall_metric    | BinaryRecall    | 0     
4 | f1score_metric   | BinaryF1Score   | 0     
-----------------------------------------------------
785       Trainable params
0         Non-trainable params
785       Total params
0.003     

Epoch 0:   0%|          | 7/12000 [00:00<02:47, 71.43it/s, v_num=xd39]

C:\Users\pc\AppData\Local\Temp\ipykernel_10292\2070092589.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)
C:\Users\pc\AppData\Local\Temp\ipykernel_10292\2070092589.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)
C:\Users\pc\AppData\Local\Temp\ipykernel_10292\2070092589.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)
C:\Users\pc\AppData\Local\Temp\ipykernel_10292\2070092589.py:23: UserWarning: To copy construct f

RuntimeError: all elements of input should be between 0 and 1